In [1]:
%set_env PYTORCH_ENABLE_MPS_FALLBACK=1

env: PYTORCH_ENABLE_MPS_FALLBACK=1


In [5]:
import pandas as pd

from datasetsforecast.long_horizon import LongHorizon

from neuralforecast.core import NeuralForecast
from neuralforecast.models import XLinear
from neuralforecast.losses.pytorch import MSE, MAE

from utilsforecast.losses import mae, mse
from utilsforecast.evaluation import evaluate

In [3]:
def load_data(name):
    if name == "ettm1":
        Y_df, exog_df, static_df = LongHorizon.load(directory='./', group='ETTm1')
        Y_df['ds'] = pd.to_datetime(Y_df['ds'])
        val_size = 11520
        test_size = 11520
        freq = '15T'
    elif name == "ettm2":
        Y_df, *_ = LongHorizon.load(directory='./', group='ETTm2')
        Y_df['ds'] = pd.to_datetime(Y_df['ds'])
        val_size = 11520
        test_size = 11520
        freq = '15T'
    elif name == 'etth1':
        Y_df, exog_df, static_df = LongHorizon.load(directory='./', group='ETTh1')
        Y_df['ds'] = pd.to_datetime(Y_df['ds'])
        val_size = 2880
        test_size = 2880
        freq = 'H'
    elif name == "etth2":
        Y_df, *_ = LongHorizon.load(directory='./', group='ETTh2')
        Y_df['ds'] = pd.to_datetime(Y_df['ds'])
        val_size = 2880
        test_size = 2880
        freq = 'H'

    return Y_df, val_size, test_size, freq, exog_df, static_df

In [4]:
Y_df, val_size, test_size, freq, exog_df, _ = load_data('ettm1')
horizon = 96

In [10]:
models = [
    XLinear(
        h=horizon, 
        input_size=horizon, 
        n_series=7,
        hidden_size=512,
        temporal_ff=256,
        channel_ff=21,
        head_dropout=0.5,
        embed_dropout=0.2,
        learning_rate=1e-4,
        batch_size=32,
        max_steps=2000, 
        early_stop_patience_steps=3),
]

nf = NeuralForecast(models=models, freq=freq)
nf_preds = nf.cross_validation(df=Y_df, val_size=val_size, test_size=test_size, n_windows=None)
nf_preds = nf_preds.reset_index()

ettm1_evaluation = evaluate(df=nf_preds, metrics=[mae, mse], models=['XLinear'], agg_fn="mean")

Seed set to 1
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name            | Type          | Params | Mode 
----------------------------------------------------------
0 | loss            | MAE           | 0      | train
1 | padder_train    | ConstantPad1d | 0      | train
2 | scaler          | TemporalNorm  | 0      | train
3 | projection      | Sequential    | 49.7 K | train
4 | temporal_gating | GatingBlock   | 525 K  | train
5 | channel_gating  | GatingBlock   | 623    | train
6 | head            | Sequential    | 98.4 K | train
  | other params    | n/a           | 3.6 K  | n/a  
----------------------------------------------------------
677 K     Trainable params
0         Non-trainable params
677 K     Total params
2.711     Total estimated model params size (MB)
23        Modules in train mode
0         Modules in eval mode


Sanity Checking DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]

/Users/marcopeix/dev/neuralforecast/.venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=10` in the `DataLoader` to improve performance.


/Users/marcopeix/dev/neuralforecast/.venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=10` in the `DataLoader` to improve performance.
/Users/marcopeix/dev/neuralforecast/.venv/lib/python3.12/site-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 1299: 100%|██████████| 1/1 [00:01<00:00,  0.78it/s, v_num=1430, train_loss_step=0.375, train_loss_epoch=0.375, valid_loss=0.406]

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/marcopeix/dev/neuralforecast/.venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:433: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=10` in the `DataLoader` to improve performance.



Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  1.61it/s]


In [11]:
ettm1_evaluation

,metric,XLinear
0,mae,0.351081
1,mse,0.326818
